In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import json

In [2]:
spark = SparkSession.builder.getOrCreate()

In [111]:
config = spark.read.option("multiline","true").json('config.json')
config_df = eval(config.toJSON().collect()[0])

In [112]:
config_df

{'character_check': "[('First Name',40),('Last Name',80)]",
 'validate_null': "['First Name','Last Name']"}

In [113]:
config_df.items()

dict_items([('character_check', "[('First Name',40),('Last Name',80)]"), ('validate_null', "['First Name','Last Name']")])

In [94]:
eval(config_df['validate_null'])

['First Name', 'Last Name']

In [103]:
list(config_df['character_check'].items())

[('First Name', 40), ('Last Name', 80)]

In [ ]:
config_df['character_check']

In [110]:
list(map(eval("validate_null"),eval(config_df['validate_null'])))

[False, False]

In [117]:
list(map(eval("character_check"),eval(config_df['character_check'])))

[True, True]

In [108]:
list(map(eval('character_check'),list(config_df['character_check'].items())))

[True, True]

In [119]:
config_df.keys()

dict_keys(['character_check', 'validate_null'])

In [121]:
for keys in config_df.keys():
    print(keys)
    print(list(map(eval(keys),eval(config_df[keys]))))

character_check
[True, True]
validate_null
[False, False]


In [39]:
df = spark.read.csv(r'Templates/User - Import Template.csv', header=True)

In [40]:
df.show(truncate=False)

+--------------------------------+----------+----------+
|Email                           |First Name|Last Name |
+--------------------------------+----------+----------+
|jane.doe@bcgasample.com         |Jane      |Doe       |
|alexis.washington@bcgasample.com|Alexis    |Washington|
|reginald.anderson@bcgasample.com|Reginald  |Anderson  |
|tatiana.smithson@bgcasample.com |Tatiana   |Smithson  |
|james.sample@bcgasample.com     |James     |Sample    |
+--------------------------------+----------+----------+



In [109]:
def validate_null(field):
    if df.filter(df[field].isNull()).first():
        return True
    else:
        return False

In [42]:
def check_email_format(field):
    regex = r'^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$' 
    df1 = df.filter(df[field].rlike(regex))
    df2 = df.subtract(df1)
    if df2.first():
        return False
    else:
        return True

In [107]:
def character_check(field_value):
    if df.filter(length(df[field_value[0]])>field_value[1]).first():
        return False
    else:
        return True

In [98]:
def validate_null(field):
    return df.select(df[field].isNotNull())

In [51]:
def character_check(field, value):
    return df.select(length(df[field])>value)

In [71]:
character_check('First Name',40).distinct().show()

+-------------------------+
|(length(First Name) > 40)|
+-------------------------+
|                    false|
+-------------------------+



In [70]:
validate_null('First Name').distinct().show()

+------------------------+
|(First Name IS NOT NULL)|
+------------------------+
|                    true|
+------------------------+



In [109]:
if validate_null('First Name') == False and validate_null('Last Name') == False and check_email_format('Email') == True:
    print('valid file')
else:
    print('invalid file')

valid file
